# Correlation between CNV event and mutation status of frequently mutated genes

In [1]:
import pandas as pd
import numpy as np
import os
import cptac
import cptac.utils as ut
import altair as alt
import scipy.stats
import statsmodels.stats.multitest
import cnvutils
import warnings

In [2]:
dss = {
    "ccrcc": cptac.Ccrcc,
    "endometrial": cptac.Endometrial,
    "hnscc": cptac.Hnscc,
    "lscc": cptac.Lscc,
    "luad": cptac.Luad,
}

In [3]:
def test_mut_correlation(
    cancer_type,
    dss,
    mut_genes=[
        "TP53",
        "PTEN",
        "KRAS",
        "PIK3CA",
        "ARID1A",
    ],
    mut_freq_cutoff=0.1,
):
    # Load the dataset
    ds = dss[cancer_type]()
    
    # Run freq_mut, and get all genes with mutation frequency above cutoff
    freqs = ut.get_frequently_mutated(ds, cutoff=mut_freq_cutoff)
    freq_mut_genes = freqs["Gene"]
    
    # Select just our desired genes
    sel_genes = [gene for gene in mut_genes if gene in freq_mut_genes.values]
    exclude_genes = [gene for gene in mut_genes if gene not in freq_mut_genes.values]
    
    if len(exclude_genes) > 0:
        warnings.warn(
            f"The following genes did not pass the {mut_freq_cutoff} mutation frequency cutoff in {cancer_type}:\n{exclude_genes}"
        )
    
    # Get the mutations table, in a format that will be easy to join to
    mut = ds.join_metadata_to_mutations(
        metadata_df_name="clinical",
        mutations_genes=sel_genes,
        metadata_cols=[], # This will format the table in the way we want
        mutations_filter=[],
        how="right",
        quiet=True,
        tissue_type="tumor"
    )
    
    # Get just the mutation status columns, and create a binarized version of them
    mut_status = mut.loc[:, mut.columns.isin([gene + "_Mutation_Status" for gene in sel_genes])]
    mut_binary = pd.DataFrame(index=mut_status.index.copy())
    
    for col in mut_status:
        assert not pd.isnull(mut_status[col]).any()
        mut_binary = mut_binary.assign(**{col: np.where(mut_status[col] == "Wildtype_Tumor", False, True)})
        
    # Get the residuals table
    res = pd.read_csv(f"{cancer_type}_residuals.tsv.gz", sep="\t", index_col=0)
    
    pvals = []
    gene_col_labels = []
    cnv_col_labels = []
    freqs_col = []

    for mut_col in mut_binary.columns:
        
        # Get above line status for patients for this gene
        gene_name = mut_col.split("_", maxsplit=1)[0]
        above_reg_line = res[res["Gene"] == gene_name]["above_reg_line"]

        # Create contingency table
        contingency_table = pd.crosstab(above_reg_line, mut_binary[mut_col])
        print(f"\n{cancer_type}, {gene_name}\n")
        print(contingency_table)
        
        # Append labels for this row
        gene_col_labels.append(mut_col)
        cnv_col_labels.append("above_reg_line")
        freqs_col.append(freqs[freqs["Gene"] == gene_name]["Unique_Samples_Mut"].iloc[0])

        # Run test
        if len(above_reg_line) > 0: # Sometimes a gene didn't get included in the residuals calculations
            chi2, p, dof, exp_freq = scipy.stats.chi2_contingency(contingency_table)
            
            # Check assumptions: No group has expected value < 1, and no more than
            # 20% of groups have expected frequency < 5.
            exp_freq = pd.DataFrame(exp_freq)

            if (exp_freq < 1).any().any():
                pvals.append(np.nan)
            elif (exp_freq < 5).sum().sum() > 0.2 * exp_freq.shape[0] * exp_freq.shape[1]:
                pvals.append(np.nan)
            else:
                pvals.append(p)
        else:
            pvals.append(np.nan)
        
    pvals = pd.DataFrame({
        "cancer_type": cancer_type,
        "cnv_event": cnv_col_labels,
        "gene": gene_col_labels,
        "pval": pvals,
        "mut_freq": freqs_col,
    })
    
    return pvals

In [4]:
all_results = pd.DataFrame()

for cancer_type in dss.keys():
    cancer_res = test_mut_correlation(cancer_type=cancer_type, dss=dss, mut_freq_cutoff=0)
    all_results = all_results.append(cancer_res)
    all_results = all_results.reset_index(drop=True)

/home/caleb/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: The following genes did not pass the 0 mutation frequency cutoff in ccrcc:
['KRAS']



ccrcc, TP53

TP53_Mutation_Status  False  True
above_reg_line                   
False                    15     1
True                     10     3

ccrcc, PTEN

Empty DataFrame
Columns: []
Index: []

ccrcc, PIK3CA

Empty DataFrame
Columns: []
Index: []

ccrcc, ARID1A

Empty DataFrame
Columns: []
Index: []
                                                
endometrial, TP53

Empty DataFrame
Columns: []
Index: []

endometrial, PTEN

Empty DataFrame
Columns: []
Index: []

endometrial, KRAS

Empty DataFrame
Columns: []
Index: []

endometrial, PIK3CA

Empty DataFrame
Columns: []
Index: []

endometrial, ARID1A

Empty DataFrame
Columns: []
Index: []


/home/caleb/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: The following genes did not pass the 0 mutation frequency cutoff in hnscc:
['KRAS']



hnscc, TP53

TP53_Mutation_Status  False  True
above_reg_line                   
False                     6    43
True                      4    44

hnscc, PTEN

PTEN_Mutation_Status  False  True
above_reg_line                   
False                    53     2
True                     54     0

hnscc, PIK3CA

PIK3CA_Mutation_Status  False  True
above_reg_line                     
False                      54     3
True                       44     8

hnscc, ARID1A

Empty DataFrame
Columns: []
Index: []
                                         
lscc, TP53

TP53_Mutation_Status  False  True
above_reg_line                   
False                     5    56
True                      0    47

lscc, PTEN

PTEN_Mutation_Status  False  True
above_reg_line                   
False                    48     9
True                     48     3

lscc, KRAS

Empty DataFrame
Columns: []
Index: []

lscc, PIK3CA

PIK3CA_Mutation_Status  False  True
above_reg_line                     
False    

/home/caleb/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: The following genes did not pass the 0 mutation frequency cutoff in luad:
['PTEN']



luad, TP53

TP53_Mutation_Status  False  True
above_reg_line                   
False                    23    18
True                     13    29

luad, KRAS

KRAS_Mutation_Status  False  True
above_reg_line                   
False                    28    23
True                     45     9

luad, PIK3CA

PIK3CA_Mutation_Status  False  True
above_reg_line                     
False                      48     3
True                       58     0

luad, ARID1A

Empty DataFrame
Columns: []
Index: []


In [5]:
all_results

,cancer_type,cnv_event,gene,pval,mut_freq
0,ccrcc,above_reg_line,TP53_Mutation_Status,NaN,0.054545
1,ccrcc,above_reg_line,PTEN_Mutation_Status,NaN,0.045455
2,ccrcc,above_reg_line,PIK3CA_Mutation_Status,NaN,0.036364
3,ccrcc,above_reg_line,ARID1A_Mutation_Status,NaN,0.009091
4,endometrial,above_reg_line,TP53_Mutation_Status,NaN,0.221053
5,endometrial,above_reg_line,PTEN_Mutation_Status,NaN,0.789474
6,endometrial,above_reg_line,KRAS_Mutation_Status,NaN,0.326316
7,endometrial,above_reg_line,PIK3CA_Mutation_Status,NaN,0.494737
8,endometrial,above_reg_line,ARID1A_Mutation_Status,NaN,0.452632
9,hnscc,above_reg_line,TP53_Mutation_Status,NaN,0.864865


In [6]:
all_results_filtered = all_results[all_results["pval"].notna()].reset_index()

## Multiple testing correction

In [7]:
reject, pvals_corrected, alphacSidak, alphacBonf = statsmodels.stats.multitest.multipletests(
    pvals=all_results_filtered["pval"], 
    alpha=0.05, 
    method="fdr_bh"
)

all_results_filtered = all_results_filtered.assign(adj_p=pvals_corrected)

In [8]:
all_results_filtered[all_results_filtered["adj_p"] <= 0.05]

,index,cancer_type,cnv_event,gene,pval,mut_freq,adj_p
5,19,luad,above_reg_line,KRAS_Mutation_Status,0.003166,0.3,0.018993


In [9]:
alt.Chart(all_results_filtered).mark_bar().encode(
    x=alt.X(
        "adj_p",
        bin=alt.Bin(step=0.05)
    ),
    y=alt.Y(
        "count()"
    )
)

alt.Chart(...)

In [10]:
all_results_filtered

,index,cancer_type,cnv_event,gene,pval,mut_freq,adj_p
0,11,hnscc,above_reg_line,PIK3CA_Mutation_Status,0.151603,0.099099,0.275842
1,14,lscc,above_reg_line,PTEN_Mutation_Status,0.183894,0.103448,0.275842
2,16,lscc,above_reg_line,PIK3CA_Mutation_Status,0.609281,0.094828,0.609281
3,17,lscc,above_reg_line,ARID1A_Mutation_Status,0.300128,0.120690,0.360154
4,18,luad,above_reg_line,TP53_Mutation_Status,0.036658,0.536364,0.109973
5,19,luad,above_reg_line,KRAS_Mutation_Status,0.003166,0.300000,0.018993


In [11]:
def pval_plot(df, title, group_col, val_col, y=True, sig=0.05):
    
    val_log_col = "neg_log_p"
    log_cutoff = -np.log10(sig)
    df = df.assign(**{val_log_col: - np.log10(df[val_col])})
    
    if y:
        chart_y = alt.Y(
            val_log_col + ":Q",
            title="-log(p)",
        )
        
    else:
        chart_y = alt.Y(
            val_log_col,
            axis=alt.Axis(
                labels=False,
                ticks=False,
                title=None
            )
        )
        
        
    chart = alt.Chart(df).mark_point().encode(
        x=group_col + ":N",
        y=chart_y,
        color=group_col
    )
    
#     chart_text = chart.transform_filter(
#         alt.datum.neg_log_p >= log_cutoff
#     ).mark_text(
#         align="left",
#         baseline='middle',
#         dx=7
#     ).encode(
#         text='gene'
#     )

    line = alt.Chart(pd.DataFrame({
        'y': [log_cutoff],
        "label": [f"-log({sig})"]
    })).mark_rule(color="crimson").encode(
        y="y"
    )

    text = line.mark_text(
        align="right",
        dx=-65
    ).encode(
        text="label"
    )
        
        
    if y:
        return (chart + line + text).properties(title=title)
    else:
        return (chart + line).properties(title=title)
    
alt.hconcat(
    pval_plot(
        all_results_filtered[all_results_filtered["cnv_event"] == "8p_loss"], "8p loss", "cancer_type", "adj_p"),
    pval_plot(
        all_results_filtered[all_results_filtered["cnv_event"] == "8q_gain"], "8q gain", "cancer_type", "adj_p", False)
    
).resolve_scale(y="shared").properties(
    title=["Chi-squared results for correlation of chr8 CNV events", "with frequently mutated genes (>10%)"]
).configure_title(
    anchor="middle"
)

/home/caleb/anaconda3/envs/dev/lib/python3.7/site-packages/altair/utils/core.py:187: UserWarning: I don't know how to infer vegalite type from 'empty'.  Defaulting to nominal.
  "Defaulting to nominal.".format(typ)


alt.HConcatChart(...)

## T-test for CNV data (EGFR)

In [12]:
cnv_genes = ["EGFR"]